In [1]:
import pandas as pd
from decouple import config
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

In [2]:
class CriarTabelasNeo4j:
    def __init__(self):
        self.uri = config('NEO4J_URI')
        self.user = config('NEO4J_USERNAME')
        self.password = config('NEO4J_PASSWORD')
        self._driver = GraphDatabase.driver(self.uri, auth=(self.user, self.password))

    def close(self):
        self._driver.close()

    def create_tables(self):
        with self._driver.session() as session:
            session.execute_write(self._create_region_table, nome_regiao="Nome da Região")
            session.execute_write(self._create_uf_table, uf="UF")
            session.execute_write(self._create_municipios_table, uf="UF", nome_municipio="Nome do Município")
            session.execute_write(self._create_beneficiarios_table)
            session.execute_write(self._create_ies_table)
            session.execute_write(self._create_cursos_table)
            session.execute_write(self._create_bolsas_table)

    @staticmethod
    def _create_region_table(tx, nome_regiao):
        query = "CREATE (r:Regiao {nome_regiao: $nome_regiao})"
        tx.run(query, nome_regiao=nome_regiao)

    @staticmethod
    def _create_uf_table(tx, uf):
        query = "CREATE (uf:UF {uf: $uf})"
        tx.run(query, uf=uf)

    @staticmethod
    def _create_municipios_table(tx, uf, nome_municipio):
        query = """
        MATCH (uf:UF {uf: $uf})
        CREATE (m:Municipio {nome_municipio: $nome_municipio, uf_id: id(uf)})
        """
        tx.run(query, uf=uf, nome_municipio=nome_municipio)

    @staticmethod
    def _create_beneficiarios_table(tx):
        query = """
        CREATE (b:Beneficiario {
            cpf: $cpf,
            sexo: $sexo,
            raca: $raca,
            data_nascimento: $data_nascimento,
            deficiente_fisico: $deficiente_fisico
        })
        """
        tx.run(query, cpf="CPF_BENEFICIARIO", sexo="SEXO_BENEFICIARIO", raca="RACA_BENEFICIARIO",
               data_nascimento="DATA_NASCIMENTO", deficiente_fisico="BENEFICIARIO_DEFICIENTE_FISICO")

    @staticmethod
    def _create_ies_table(tx):
        query = """
        CREATE (ies:IES {
            codigo_emec: $codigo_emec,
            nome: $nome
        })
        """
        tx.run(query, codigo_emec="CODIGO_EMEC_IES_BOLSA", nome="NOME_IES_BOLSA")

    @staticmethod
    def _create_cursos_table(tx):
        query = """
        CREATE (c:Curso {
            nome_curso: $nome_curso,
            turno: $turno
        })
        """
        tx.run(query, nome_curso="NOME_CURSO_BOLSA", turno="NOME_TURNO_CURSO_BOLSA")

    @staticmethod
    def _create_bolsas_table(tx):
        query = """
        CREATE (bo:Bolsa {
            ano_concessao: $ano_concessao,
            tipo_bolsa: $tipo_bolsa,
            modalidade_ensino: $modalidade_ensino
        })
        """
        tx.run(query, ano_concessao="ANO_CONCESSAO_BOLSA", tipo_bolsa="TIPO_BOLSA", modalidade_ensino="MODALIDADE_ENSINO_BOLSA")


In [3]:
# Crie uma instância da classe Neo4jDataLoader
neo4j_creator = CriarTabelasNeo4j()

# Carregue o DataFrame com os dados
dataframe = pd.read_csv("../data/processed/ProuniRelatorio_processed.csv", encoding="UTF-8")

# Execute o método para carregar dados no Neo4j
neo4j_creator.create_tables()
neo4j_creator.close()